# Imports

In [2]:
# Std lib
from collections import namedtuple, Counter, defaultdict
from itertools import islice
import random
import glob
from pprint import pprint as pp

# Third party
import numpy as np
import pandas as pd
from matplotlib import pyplot as pl
from pycl.pycl import *
from tqdm import tqdm, tqdm_notebook

# Matplotlib and pandas setup
%matplotlib inline
pd.options.display.max_colwidth = 200
pd.options.display.max_columns = 200

# Dev whitelist function

In [76]:
head ("/home/aleg/Analyses/Nanopore_yeast/nanopolish/nanopolish_read_raw_collapsed.tsv.idx")

ref_id  ref_start ref_end read_id                              kmers NNNNN_kmers mismatching_kmers missing_kmers offset 
YHR055C 0         165     7ef1d7b9-5824-4382-b23b-78d82c07ebbd 155   6           0                 10            0      
YHR055C 1         182     68392b1f-4591-4917-8d56-53f3a8003cd6 178   3           0                 3             10478  
YHR055C 0         182     be088f2c-0c1a-434c-bd32-a8a516da476a 165   10          0                 17            22534  
YHR055C 3         171     eba44c90-5209-4216-8b75-67b7abd0a25f 160   6           0                 8             33722  
YHR055C 0         182     cf54e40f-af41-4450-a32c-584d8bd3204d 165   10          0                 17            44557  
YOL138C 3609      4019    e0701bb5-012d-4f1e-8043-8a12dcaa49ab 390   13          0                 20            55794  
YPR165W 293       626     ca16553b-fbd6-4539-8a4d-9826340eb380 306   16          0                 27            82643  
YOL138C 3131      4015    609b08

In [66]:
def whitelist (
    s1_fn,
    s2_fn,
    fasta_index_fn = None,
    min_cov = 10,
    max_NNNNN_kmers_freq = 0.2,
    max_mismatching_kmers_freq = 0.2,
    max_missing_kmers_freq = 0.2,
    verbose=True):

    # Read fasta index
    if verbose: print ("Read fasta index")
    ref_len_dict = _read_fasta_index (fn=fasta_index_fn)
    if verbose: print (f"\tTotal references: {len(ref_len_dict)}")

    # Create reference index for both files
    if verbose: print ("Read eventalign index files")
    ref_reads = _read_eventalign_index (s1_fn, s2_fn, max_NNNNN_kmers_freq, max_mismatching_kmers_freq, max_missing_kmers_freq)
    if verbose: print (f"\tTotal references found {len(ref_reads)}")
    
    # Intersect both samples
    if verbose: print ("Filter out references with low coverage")
    ref_reads = _select_ref (ref_reads=ref_reads, min_cov=min_cov)
    if verbose: print (f"\tTranscripts remaining after reference coverage filtering: {len(ref_reads)}")
    
    if verbose: print ("Compute coverage per reference and select intervals with high enough coverage")
    ref_interval_reads = OrderedDict ()
    for ref_id, sample_reads in ref_reads.items ():
        # Compute reference coverage
        cov_array = _compute_ref_cov (sample_reads=sample_reads, ref_len=ref_len_dict[ref_id])
        # Get coordinates of intervals with minimum coverage
        valid_interval_list = _get_valid_intervals (cov_array, min_cov)
        # Intesect reads with valid coverage for both samples
        ref_interval_reads [ref_id] = _intersect_reads_interval (valid_interval_list, sample_reads)
    
    return ref_interval_reads
        
def _read_fasta_index (fn):
    ref_len = OrderedDict ()
    with open (fn) as fp:
        for line in fp:
            ls = line.rstrip().split()
            ref_len[ls[0]] = int(ls[1])
    return ref_len

def _read_eventalign_index (s1_fn, s2_fn, max_NNNNN_kmers_freq, max_mismatching_kmers_freq, max_missing_kmers_freq):
    ref_reads = OrderedDict ()
    
    for lab, fn in ("S1", s1_fn), ("S2", s2_fn):
        with open (fn) as fp:
            # get field names from header
            header = fp.readline().rstrip().split()
            line_tuple = namedtuple("line_tuple", header)
            c = Counter ()
            for line in fp:
                ls = line.rstrip().split()
                lt = line_tuple (ls[0], int(ls[1]), int(ls[2]), ls[3], int(ls[4]), int(ls[5]) , int(ls[6]) , int(ls[7]) , int(ls[8]))
                # filter out reads with high number of problematic kmers
                if max_NNNNN_kmers_freq and lt.NNNNN_kmers/lt.kmers > max_NNNNN_kmers_freq:
                    c ["high NNNNN_kmers reads"] += 1
                elif max_mismatching_kmers_freq and lt.mismatching_kmers/lt.kmers > max_mismatching_kmers_freq:
                    c ["high mismatching_kmers reads"] += 1
                elif max_missing_kmers_freq and lt.missing_kmers/lt.kmers > max_missing_kmers_freq:
                    c ["high missing_kmers reads"] += 1
                # Save valid reads
                else:
                    if not lt.ref_id in ref_reads:
                        ref_reads[lt.ref_id] = OrderedDict ()
                    if not lab in ref_reads [lt.ref_id]:
                        ref_reads[lt.ref_id][lab] = []
                    ref_reads[lt.ref_id][lab].append (lt)
                    c ["valid reads"] += 1
        print (c)
    return ref_reads

def _select_ref (ref_reads, min_cov):
    invalid_ref = []
    for ref_id, sample_reads in ref_reads.items ():
        if len(sample_reads) < 2 or len (sample_reads["S1"]) < min_cov or len (sample_reads["S2"]) < min_cov:
            invalid_ref.append (ref_id)
    for ref_id in invalid_ref:
        del ref_reads [ref_id]
    return ref_reads

def _compute_ref_cov (sample_reads, ref_len):
    cov_array = np.zeros ((2, ref_len))
    for read in sample_reads["S1"]:
        cov_array [0][np.arange(read.ref_start, read.ref_end)] += 1
    for read in sample_reads["S2"]:
        cov_array [1][np.arange(read.ref_start, read.ref_end)] += 1
    return cov_array
        
def _get_valid_intervals (cov_array, min_cov):
    valid_cov = False
    valid_interval_list = []
    for pos, (cov1, cov2) in enumerate (cov_array.T):
        # If coverage insuficient
        if cov1 < min_cov or cov2 < min_cov:
            if valid_cov:
                valid_interval_list.append ((ref_start, ref_end))
            valid_cov = False
        # If the coverage is high enough for both samples
        else:
            if valid_cov:
                ref_end = pos
            else:
                ref_start = ref_end = pos
                valid_cov = True
    # Last valid interval exception
    if valid_cov:
        valid_interval_list.append ((ref_start, ref_end))
    
    return valid_interval_list

def _intersect_reads_interval (valid_interval_list, sample_reads):
    
    ref_interval_reads = OrderedDict ()
    for interval_start, interval_end in valid_interval_list:
        ref_interval_reads [(interval_start, interval_end)] = {"S1":[], "S2":[]}
    
    for sample_id, read_list in sample_reads.items():
        for read in read_list:
            for interval_start, interval_end in valid_interval_list: 
                if read.ref_end >= interval_start and read.ref_start <= interval_end:
                    ref_interval_reads[(interval_start, interval_end)][sample_id].append (read)
    return ref_interval_reads

In [75]:
sample1_fn = "/home/aleg/Analyses/RNA_Yeast_TRM5/eventalign/KO/eventalign_collapsed.tsv.idx"
sample2_fn = "/home/aleg/Analyses/RNA_Yeast_TRM5/eventalign/WT/eventalign_collapsed.tsv.idx"
fasta_index_fn = "/home/aleg/Analyses/Nanopore_yeast/references/SC_R64-1-1_transcripts.fa.fai"
w  = whitelist (sample1_fn, sample2_fn, fasta_index_fn, min_cov=5, max_NNNNN_kmers_freq=0.1, max_mismatching_kmers_freq=0.1, max_missing_kmers_freq=0.1)

Read fasta index
	Total references: 6713
Read eventalign index files
Counter({'valid reads': 11073, 'high missing_kmers reads': 927, 'high mismatching_kmers reads': 344, 'high NNNNN_kmers reads': 84})
Counter({'valid reads': 10802, 'high missing_kmers reads': 702, 'high mismatching_kmers reads': 384, 'high NNNNN_kmers reads': 84})
	Total references found 2126
Filter out references with low coverage
	Transcripts remaining after reference coverage filtering: 172
Compute coverage per reference and select intervals with high enough coverage


# Test whitelist from package

In [1]:
from nanocompore.Whitelist import Whitelist

In [6]:
sample1_fn = "/home/aleg/Analyses/RNA_Yeast_TRM5/eventalign/KO/eventalign_collapsed.tsv.idx"
sample2_fn = "/home/aleg/Analyses/RNA_Yeast_TRM5/eventalign/WT/eventalign_collapsed.tsv.idx"
fasta_index_fn = "/home/aleg/Analyses/RNA_Yeast_TRM5/references/SC_R64-1-1_transcripts_clean.fa.fai"
w  = Whitelist (sample1_fn, sample2_fn, fasta_index_fn,
                min_coverage=5,
                max_coverage=100,
                max_NNNNN_kmers_freq=0.1,
                max_mismatching_kmers_freq=0.1,
                max_missing_kmers_freq=0.1,
                logLevel="debug")

Initialise and checks options
Read fasta index files
	Total references: 6713
Read eventalign index files
	Sample S1 	valid reads: 11,073	high missing_kmers reads: 927	high mismatching_kmers reads: 344	high NNNNN_kmers reads: 84
	Sample S2 	valid reads: 10,802	high missing_kmers reads: 702	high mismatching_kmers reads: 384	high NNNNN_kmers reads: 84
	References found in index: 2126
Filter out references with low coverage
	positions: 181,719	S1_reads: 8,941	S2_reads: 7,889	ref_id: 172
	References remaining after reference coverage filtering: 172
Compute coverage per reference and select intervals with high enough coverage
100%|██████████| 172/172 [00:00<00:00, 523.80 References/s]
	positions: 67,876	S2_reads: 4,558	S1_reads: 2,402	intervals: 172	ref_id: 166
	References remaining after position coverage filtering: 166


In [4]:
for ref_id, ref_dict in islice(w.ref_interval_reads.items(), 10000):
    if len(ref_dict["interval_list"]) >=2: 
        jprint (ref_id, bold=True, size=150)
        pp (ref_dict["interval_list"])
        with pd.option_context("display.max_rows", 4):
            display (pd.DataFrame(ref_dict["S1"]))
            display (pd.DataFrame(ref_dict["S2"]))

[(256, 291), (328, 937)]


,ref_id,ref_start,ref_end,read_id,kmers,NNNNN_kmers,mismatching_kmers,missing_kmers,offset
0,YGR282C,68,292,f31e522b-8cb5-41ea-9fbe-919b7dfa2c22,211,13,0,13,14778454
1,YGR282C,59,836,39d52c27-39c5-4cf4-80f4-e4a33ccd5341,758,18,0,19,14792452
...,...,...,...,...,...,...,...,...,...
21,YGR282C,785,938,2e99fc24-4f5e-4074-b888-f52e7ec8fc57,144,6,0,9,15379603
22,YGR282C,800,930,abbd6905-183f-41a3-b69d-ed051c24e2cc,128,2,0,3,15389484


,ref_id,ref_start,ref_end,read_id,kmers,NNNNN_kmers,mismatching_kmers,missing_kmers,offset
0,YGR282C,1,938,b7e6ba90-eda3-49b1-9901-8ce7b82607f6,909,25,0,28,35886978
1,YGR282C,1,931,ed294476-9d30-4db3-9998-08e040e2d02e,884,30,0,46,35948292
...,...,...,...,...,...,...,...,...,...
14,YGR282C,49,938,e3ef4d16-352d-4563-a577-5de85b078ca4,842,32,0,47,36622513
15,YGR282C,642,938,09cbcc5a-7e53-49d8-9267-5e5395bc99ed,278,12,0,18,36679904


[(428, 479), (515, 752)]


,ref_id,ref_start,ref_end,read_id,kmers,NNNNN_kmers,mismatching_kmers,missing_kmers,offset
0,YGR214W,0,755,ad3944d6-56af-4588-adb6-b092977063f4,725,18,0,30,17215061
1,YGR214W,283,755,4c537718-3cf8-4f5e-b5ad-6a27a91b5294,446,11,0,26,17263729
...,...,...,...,...,...,...,...,...,...
5,YGR214W,515,753,f85c6169-a90c-41c7-a038-1192c79967da,231,5,0,7,17408168
6,YGR214W,591,755,f057fbd9-5c54-4aa1-bde0-1965cb0208b5,157,6,0,7,17423948


...

,ref_id,ref_start,ref_end,read_id,kmers,NNNNN_kmers,mismatching_kmers,missing_kmers,offset
0,YER138C,3692,4276,384d007f-2a40-479c-9475-d137c600a04b,542,18,0,42,384758032
1,YER138C,1561,2050,946f84b1-bf7d-4843-8b3b-870b7622ba2d,450,22,0,39,384795087
...,...,...,...,...,...,...,...,...,...
7,YER138C,4949,5264,fc53b0a7-5bbe-4b35-9244-cd6df7ce4267,302,12,0,18,385070463
8,YER138C,4809,5264,21abc37e-9d1f-4838-b8d3-9537d6a12fa6,440,12,0,15,385105908


,ref_id,ref_start,ref_end,read_id,kmers,NNNNN_kmers,mismatching_kmers,missing_kmers,offset
0,YER138C,3039,5260,6452c5d7-c5fc-4fa7-9019-e9add22558eb,2055,97,0,166,399062802
1,YER138C,1822,4443,67705ada-e198-4e0c-8b7d-003f66daf460,2540,61,0,81,399203633
...,...,...,...,...,...,...,...,...,...
5,YER138C,5033,5264,22e6a767-bcd0-4a02-b832-871cbc972779,218,5,0,13,399716252
6,YER138C,3536,5264,7f9ccf05-8579-4327-9eb8-d51507f11de5,1659,49,0,69,399731355


[(4809, 4912), (5038, 5102)]


,ref_id,ref_start,ref_end,read_id,kmers,NNNNN_kmers,mismatching_kmers,missing_kmers,offset
0,YOL103W-B,1406,1995,6077e2ef-cf62-424b-b115-e6de3d79584a,571,15,0,18,405557881
1,YOL103W-B,3692,4276,384d007f-2a40-479c-9475-d137c600a04b,542,18,0,42,405716166
...,...,...,...,...,...,...,...,...,...
7,YOL103W-B,4809,5264,21abc37e-9d1f-4838-b8d3-9537d6a12fa6,442,12,0,13,406036594
8,YOL103W-B,5038,5259,a52a2c1b-a567-484b-935b-c44ffc2f42bf,209,9,0,15,406066990


,ref_id,ref_start,ref_end,read_id,kmers,NNNNN_kmers,mismatching_kmers,missing_kmers,offset
0,YOL103W-B,3039,5260,6452c5d7-c5fc-4fa7-9019-e9add22558eb,2059,97,0,162,460050822
1,YOL103W-B,1822,4443,67705ada-e198-4e0c-8b7d-003f66daf460,2540,61,0,81,460191924
...,...,...,...,...,...,...,...,...,...
6,YOL103W-B,3536,5264,7f9ccf05-8579-4327-9eb8-d51507f11de5,1655,49,0,73,460749756
7,YOL103W-B,5098,5257,e2253b88-3291-4085-8e6a-6d473aba8b61,149,7,0,10,460893434


In [11]:
s1_index_fn = "/home/aleg/Analyses/RNA_Yeast_TRM5/eventalign/KO/eventalign_collapsed.tsv.idx"
s2_index_fn = "/home/aleg/Analyses/RNA_Yeast_TRM5/eventalign/WT/eventalign_collapsed.tsv.idx"
fasta_index_fn = "/home/aleg/Analyses/Nanopore_yeast/references/SC_R64-1-1_transcripts.fa.fai"

w  = Whitelist (s1_index_fn, s2_index_fn, fasta_index_fn, min_coverage=15, max_NNNNN_kmers_freq=0.1, max_mismatching_kmers_freq=0.1, max_missing_kmers_freq=0.1, logLevel="debug")

Initialise and checks options
Read fasta index files
	Total references: 6713
Read eventalign index files
	Sample S1 	valid reads: 11,073	high missing_kmers reads: 927	high mismatching_kmers reads: 344	high NNNNN_kmers reads: 84
	Sample S2 	valid reads: 10,802	high missing_kmers reads: 702	high mismatching_kmers reads: 384	high NNNNN_kmers reads: 84
	References found in index: 2126
Filter out references with low coverage
	positions: 39,312	S1_reads: 7,959	S2_reads: 5,934	ref_id: 50
	References remaining after reference coverage filtering: 50
Compute coverage per reference and select intervals with high enough coverage
100%|██████████| 50/50 [00:00<00:00, 299.82 References/s]
	positions: 15,054	S1_reads: 7,841	S2_reads: 5,659	intervals: 47	ref_id: 45
	References remaining after position coverage filtering: 45


In [9]:
w.ref_id_list[:10]

['YKL096W-A',
 'YLR029C',
 'YLR249W',
 'YKR059W',
 'YOR369C',
 'YJR145C',
 'YKL006W',
 'YOL120C',
 'YNL069C',
 'YGR282C']

In [13]:
w.to_bed ("/home/aleg/Analyses/RNA_Yeast_TRM5/test/whitelist.bed")
head ("/home/aleg/Analyses/RNA_Yeast_TRM5/test/whitelist.bed")

YKL096W-A 10  272  
YLR029C   388 556  
YKR059W   878 1177 
YOR369C   21  420  
YJR145C   530 776  
YKL006W   176 230  
YKL006W   245 377  
YOL120C   238 534  
YGR282C   643 910  
YHR141C   12  311  



# nanopolish eventalign parser

In [15]:
from nanocompore.helper_lib import mytqdm
from nanocompore.Whitelist import Whitelist

class SampComp (object):
    def __init__ (self, s1_fn, s2_fn, whitelist):
        self.__whitelist = whitelist
        self.__s1_fn = s1_fn
        self.__s2_fn = s2_fn
        self.__logLevel = "info"

    def read_eventalign (self):
        
          with open (self.__s1_fn) as s1_fp, open (self.__s2_fn) as s2_fp: # Open only once
                pbar = mytqdm (total = len(self.__whitelist), unit=" References", disable=self.__logLevel == "warning")
                for ref_id, ref_dict in self.__whitelist:
                    pbar.update ()
                    
                    # Init empty dict for all positions in valid intervals
                    position_dict = OrderedDict ()
                    for interval_start, interval_end in ref_dict["interval_list"]:
                        for i in range (interval_start, interval_end+1):
                            position_dict[i] = {"S1_mean":[], "S1_dwell":[], "S2_mean":[], "S2_dwell":[]}

                    # Parse S1 and S2 reads data and add to mean and dwell time per position
                    for lab, fp in (("S1", s1_fp), ("S2", s2_fp)):
                        for read in ref_dict[lab]:

                            # Move to read save read data chunk and reset file pointer 
                            fp.seek (read.byte_offset)
                            read_lines = fp.read (read.byte_len)
                            fp.seek (0)
                    
                            # Check if positions are in the ones found in the whitelist intervals 
                            for line in read_lines.split("\n")[2:]:
                                ls = line.split("\t")
                                ref_pos = int(ls[0])
                                
                                # Append mean value and dwell time
                                if ref_pos in position_dict:
                                    position_dict[ref_pos][lab+"_mean"].append(float(ls[8]))
                                    position_dict[ref_pos][lab+"_dwell"].append(int(ls[9]))
                    
                    yield ((ref_id, position_dict))


# Test SampComp

In [25]:
# -*- coding: utf-8 -*-

#~~~~~~~~~~~~~~IMPORTS~~~~~~~~~~~~~~#
# Std lib
import logging
from collections import defaultdict, OrderedDict, namedtuple, Counter
import os
import shelve
import multiprocessing as mp

# Third party
import numpy as np

# Local package
#from nanocompore.txCompare import txCompare
from nanocompore.helper_lib import mkdir, access_file, mytqdm
from nanocompore.Whitelist import Whitelist
from nanocompore.NanocomporeError import NanocomporeError

# Logger setup
logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger(__name__)
logLevel_dict = {"debug":logging.DEBUG, "info":logging.INFO, "warning":logging.WARNING}

#~~~~~~~~~~~~~~MAIN FUNCTION~~~~~~~~~~~~~~#
class SampComp (object):
    """ Produce useful results. => Thanks Tommaso ! That's a very *useful* comment :P """

    def __init__(self,
        s1_fn,
        s2_fn,
        whitelist,
        output_db_fn,
        padj_threshold = 0.1,
        comparison_method = "kmean",
        nthreads = 4,
        logLevel = "info",):

        """
        Main routine that starts readers and consumers
            s1_fn: path to a nanopolish eventalign collapsed file corresponding to sample 1
            s2_fn: path to a nanopolish eventalign collapsed file corresponding to sample 2
            outfolder: path to folder where to save output
            nthreads: number of consumer threads
            whitelist_dict: Dictionnary generated by nanocopore whitelist function
        """
        # Set logging level
        logger.setLevel (logLevel_dict.get (logLevel, logging.WARNING))
        logger.info ("Initialise and checks options")
        
        # Check args
        if not isinstance (whitelist, Whitelist):
            raise NanocomporeError("Whitelist is not valid")
        for fn in (s1_fn, s2_fn):
            access_file (fn)
        if nthreads < 3:
            raise NanocomporeError("Number of threads not valid")
        #mkdir (outfolder)    
        
        # Save private args
        self.__logLevel = logLevel
        self.__padj_threshold = padj_threshold
        self.__whitelist = whitelist
        self.__s1_fn = s1_fn
        self.__s2_fn = s2_fn
        self.__output_db_fn = output_db_fn
        self.__nthreads = nthreads-2
        
        logger.info ("Start data processing")
        
        # Init Multiprocessing variables
        in_q = mp.Queue (maxsize = 1000)
        out_q = mp.Queue (maxsize = 1000)

        # Define processes
        ps_list = []
        ps_list.append (mp.Process (target=self._read_eventalign_files, args=(in_q,)))
        for i in range (self.__nthreads):
            ps_list.append (mp.Process (target=self._process_references, args=(in_q, out_q)))
        ps_list.append (mp.Process (target=self._write_output, args=(out_q,)))

        # Start processes and block until done
        try:
            for ps in ps_list:
                ps.start ()
            for ps in ps_list:
                ps.join ()

        # Kill processes if early stop
        except (BrokenPipeError, KeyboardInterrupt) as E:
            if self.verbose: stderr_print ("Early stop. Kill processes\n")
            for ps in ps_list:
                ps.terminate ()
    
    def _read_eventalign_files (self, in_q):      
        # Add refid to inqueue to dispatch the data among the workers
        with open (self.__s1_fn) as s1_fp, open (self.__s2_fn) as s2_fp:                
            for ref_id, ref_dict in self.__whitelist:

                # Init empty dict for all positions in valid intervals
                position_dict = OrderedDict ()
                for interval_start, interval_end in ref_dict["interval_list"]:
                    for i in range (interval_start, interval_end+1):
                        position_dict[i] = {"S1_mean":[], "S1_dwell":[], "S2_mean":[], "S2_dwell":[]}

                # Parse S1 and S2 reads data and add to mean and dwell time per position
                for lab, fp in (("S1", s1_fp), ("S2", s2_fp)):
                    for read in ref_dict[lab]:

                        # Move to read save read data chunk and reset file pointer 
                        fp.seek (read.byte_offset)
                        read_lines = fp.read (read.byte_len)
                        fp.seek (0)

                        # Check if positions are in the ones found in the whitelist intervals 
                        for line in read_lines.split("\n")[2:]:
                            ls = line.split("\t")
                            ref_pos = int(ls[0])

                            # Append mean value and dwell time
                            if ref_pos in position_dict:
                                position_dict[ref_pos][lab+"_mean"].append(float(ls[8]))
                                position_dict[ref_pos][lab+"_dwell"].append(int(ls[9]))
                    
                in_q.put ((ref_id, position_dict))
                    
        # Add 1 poison pill for each worker thread
        for i in range (self.__nthreads):
            in_q.put (None)
    
    def _process_references (self, in_q, out_q): 
        # Consumme ref_id until a poison pill is found
        for ref_id, position_dict in iter (in_q.get, None):
            # Do stats with position_dict
            ####### position_dict = tx_compare (padj_threshold, comparison_method) ## Add p-value per position to the position_dict. #######
            # Add the current read details to queue
            out_q.put ((ref_id, position_dict))
        
        # Add poison pill in queues
        out_q.put (None)
    
    def _write_output (self, out_q):
        
        with shelve.open (self.__output_db_fn) as db:
            # Iterate over the counter queue and process items until all poison pills are found
            pbar = tqdm (total = len(self.__whitelist), unit=" Processed References", disable=self.__logLevel=="warning")
            for _ in range (self.__nthreads):
                for ref_id, stat_dict in iter (out_q.get, None):
                    # Write results in a shelve db to get around multithreaded isolation
                    db [ref_id] = stat_dict 
                    pbar.update ()
            pbar.close()
                    

# Test package

In [8]:
from nanocompore.SampComp import SampComp
from nanocompore.Whitelist import Whitelist

In [10]:
w  = Whitelist (
    s1_index_fn = "/home/aleg/Analyses/RNA_Yeast_TRM5/eventalign/KO/eventalign_collapsed.tsv.idx",
    s2_index_fn = "/home/aleg/Analyses/RNA_Yeast_TRM5/eventalign/WT/eventalign_collapsed.tsv.idx",
    fasta_index_fn = "/home/aleg/Analyses/RNA_Yeast_TRM5/references/SC_R64-1-1_transcripts_clean.fa.fai",
    min_coverage=10,
    max_coverage=100)

s = SampComp (
    s1_fn = "/home/aleg/Analyses/RNA_Yeast_TRM5/eventalign/KO/eventalign_collapsed.tsv",
    s2_fn = "/home/aleg/Analyses/RNA_Yeast_TRM5/eventalign/WT/eventalign_collapsed.tsv",
    output_db_fn = "/home/aleg/Analyses/RNA_Yeast_TRM5/test/out.db",
    whitelist = w)

Initialise and checks options
Read fasta index files
	Total references: 6713
Read eventalign index files
	References found in index: 2208
Filter out references with low coverage
	References remaining after reference coverage filtering: 93
Compute coverage per reference and select intervals with high enough coverage
100%|██████████| 93/93 [00:00<00:00, 413.08 References/s]
	References remaining after position coverage filtering: 87
Initialise and checks options
Start data processing
100%|██████████| 87/87 [00:05<00:00, 16.87 Processed References/s]


[61, 97, 43, 56, 12, 4, 79, 29, 53, 10]